In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import librosa
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
from IPython.display import Audio
import numpy as np
from sklearn.dummy import DummyClassifier
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
from sklearn.metrics import confusion_matrix
import os
import sys
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization, Dense
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
import seaborn as sns
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report

In [3]:
pict = "/content/drive/MyDrive/Artfct/"
art_folders = os.listdir(pict)
art_folders.sort() 
art_folders

['000', '001', '010', '011', '100', '101', '110']

In [4]:
ball_label = []
cod_label = []
light_label = []
image_id = []
for i in art_folders:
    filename = os.listdir(pict + i)
    for f in filename:
        ball_label.append(int(i[0]))
        cod_label.append(int(i[1]))
        light_label.append(int(i[2]))
        image_id.append(pict + i + '/' + f)
art_df = pd.DataFrame(light_label)
art_df = pd.concat([pd.DataFrame(cod_label),art_df,],axis=1)
art_df = pd.concat([pd.DataFrame(ball_label),art_df,],axis=1)
art_df = pd.concat([pd.DataFrame(image_id),art_df,],axis=1)
art_df.columns = ['image_id','ball_label','cod_label','light_label']
art_df

,image_id,ball_label,cod_label,light_label
0,/content/drive/MyDrive/Artfct/000/C00002934974...,0,0,0
1,/content/drive/MyDrive/Artfct/000/C00002934635...,0,0,0
2,/content/drive/MyDrive/Artfct/000/C00002933560...,0,0,0
3,/content/drive/MyDrive/Artfct/000/C00002934702...,0,0,0
4,/content/drive/MyDrive/Artfct/000/C00002935700...,0,0,0
...,...,...,...,...
1266,/content/drive/MyDrive/Artfct/110/C00002987683...,1,1,0
1267,/content/drive/MyDrive/Artfct/110/C00002987683...,1,1,0
1268,/content/drive/MyDrive/Artfct/110/C00002987683...,1,1,0
1269,/content/drive/MyDrive/Artfct/110/C00002987648...,1,1,0


In [10]:
art_df = art_df.sample(frac=1).reset_index(drop=True)
art_df

,image_id,ball_label,cod_label,light_label
0,/content/drive/MyDrive/Artfct/010/C00002961640...,0,1,0
1,/content/drive/MyDrive/Artfct/001/C00002957536...,0,0,1
2,/content/drive/MyDrive/Artfct/010/C00002957693...,0,1,0
3,/content/drive/MyDrive/Artfct/101/C00002960756...,1,0,1
4,/content/drive/MyDrive/Artfct/001/C00002956998...,0,0,1
...,...,...,...,...
1266,/content/drive/MyDrive/Artfct/010/C00002954951...,0,1,0
1267,/content/drive/MyDrive/Artfct/010/C00002954532...,0,1,0
1268,/content/drive/MyDrive/Artfct/110/C00002944809...,1,1,0
1269,/content/drive/MyDrive/Artfct/010/C00002956595...,0,1,0


In [11]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import preprocess_input

datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                             rotation_range=30,
                             horizontal_flip=True,
                             fill_mode="nearest")

train_df = art_df[:int(0.8*len(art_df))]
valid_df = art_df[int(0.8*len(art_df)):]

In [12]:
y_columns = ["ball_label", "cod_label","light_label"]

train_generator = datagen.flow_from_dataframe(dataframe=train_df, 
                                              x_col="image_id", y_col=y_columns,
                                              class_mode="raw", 
                                              target_size=(500,500), batch_size=16,
                                              shuffle=True)
valid_generator = datagen.flow_from_dataframe(dataframe=valid_df, 
                                              x_col="image_id", y_col=y_columns,
                                              class_mode="raw", 
                                              target_size=(500,500), batch_size=16)

Found 1016 validated image filenames.
Found 255 validated image filenames.


In [15]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(layers.AveragePooling2D(pool_size=(2, 2), input_shape=(500, 500, 3)))
model.add(layers.Conv2D(64, kernel_size=(8), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(128, kernel_size=(8),activation='relu',kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(layers.AveragePooling2D(pool_size=(2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(6)))
model.add(layers.Dropout(0.4))
model.add(layers.Conv2D(128, kernel_size=(8),activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(6)))
model.add(layers.Dropout(0.4))
model.add(layers.Conv2D(256, kernel_size=(2),activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D(pool_size=(1)))
model.add(layers.Dropout(0.4))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(3, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=0.001),metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
average_pooling2d_9 (Average (None, 250, 250, 3)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 243, 243, 64)      12352     
_________________________________________________________________
batch_normalization_10 (Batc (None, 243, 243, 64)      256       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 236, 236, 128)     524416    
_________________________________________________________________
average_pooling2d_10 (Averag (None, 118, 118, 128)     0         
_________________________________________________________________
batch_normalization_11 (Batc (None, 118, 118, 128)     512       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 19, 19, 128)      

In [16]:
checkpoint = ModelCheckpoint("best_initial_model.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max', save_weights_only=True)
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit(train_generator,
          batch_size=16,
          epochs=64,
          validation_data=valid_generator,
          steps_per_epoch=STEP_SIZE_TRAIN,
          validation_steps=STEP_SIZE_VALID,
          callbacks=[checkpoint])

Epoch 1/64
63/63 [==============================] - 128s 2s/step - loss: 1.4289 - accuracy: 0.6188 - val_loss: 2.3050 - val_accuracy: 0.7042

Epoch 00001: val_accuracy improved from -inf to 0.70417, saving model to best_initial_model.hdf5
Epoch 2/64
63/63 [==============================] - 125s 2s/step - loss: 0.7848 - accuracy: 0.8069 - val_loss: 1.0391 - val_accuracy: 0.7083

Epoch 00002: val_accuracy improved from 0.70417 to 0.70833, saving model to best_initial_model.hdf5
Epoch 3/64
63/63 [==============================] - 125s 2s/step - loss: 0.4481 - accuracy: 0.8309 - val_loss: 0.4587 - val_accuracy: 0.8000

Epoch 00003: val_accuracy improved from 0.70833 to 0.80000, saving model to best_initial_model.hdf5
Epoch 4/64
63/63 [==============================] - 125s 2s/step - loss: 0.3767 - accuracy: 0.8149 - val_loss: 0.3635 - val_accuracy: 0.7917

Epoch 00004: val_accuracy did not improve from 0.80000
Epoch 5/64
63/63 [==============================] - 125s 2s/step - loss: 0.3233 

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import resnet50

base_model = ResNet50(include_top=False,
                      weights='imagenet',
                      pooling='avg')
'''
x = base_model.output
x = Dense(1000, activation='relu')(x)
x = Dropout(rate=0.2)(x)
out = Dense(3, activation='sigmoid')(x)
model = keras.Model(inputs=base_model.input, outputs=out)

base_model = ResNet50(include_top=False, 
                      weights='imagenet', 
                      input_shape=(2000, 2000, 3))
'''

model = Sequential()

model.add(layers.AveragePooling2D(pool_size=(4, 4), input_shape=(2000, 2000, 3)))
model.add(base_model)
#model.add(Flatten())
#model.add(Dense(1000, activation="relu"))
#model.add(Dropout(0.2))
model.add(Dense(3, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=0.001),metrics=['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint("best_initial_model.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max', save_weights_only=True)
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit(train_generator,
          batch_size=16,
          epochs=64,
          validation_data=valid_generator,
          steps_per_epoch=STEP_SIZE_TRAIN,
          validation_steps=STEP_SIZE_VALID,
          callbacks=[checkpoint])

Epoch 1/64
25/25 [==============================] - 759s 30s/step - loss: 0.5722 - accuracy: 0.7244 - val_loss: 842514.8125 - val_accuracy: 0.4271

Epoch 00001: val_accuracy improved from -inf to 0.42708, saving model to best_initial_model.hdf5
Epoch 2/64
25/25 [==============================] - 510s 21s/step - loss: 0.4162 - accuracy: 0.7479 - val_loss: 72867.8750 - val_accuracy: 0.1771

Epoch 00002: val_accuracy did not improve from 0.42708
Epoch 3/64
25/25 [==============================] - 502s 20s/step - loss: 0.3461 - accuracy: 0.7717 - val_loss: 14450.1299 - val_accuracy: 0.4271

Epoch 00003: val_accuracy did not improve from 0.42708
Epoch 4/64
25/25 [==============================] - 505s 20s/step - loss: 0.3472 - accuracy: 0.8000 - val_loss: 378.5050 - val_accuracy: 0.4062

Epoch 00004: val_accuracy did not improve from 0.42708
Epoch 5/64
25/25 [==============================] - 513s 21s/step - loss: 0.3499 - accuracy: 0.7768 - val_loss: 30.1032 - val_accuracy: 0.6667

Epoch 0

In [ ]:
import numpy as np

from os import listdir
from os.path import join
from skimage.io import imread

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten, Dense, 
                          Dropout, Activation, BatchNormalization)
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam, SGD

from skimage.transform import resize, rotate

AXIS_SIZE = 224
NUMBER_CLASSES = 3
EPOCHS_NUMBER = 20
BATCHS_NUMBER = 256


def resize_img(img):
    new_img = resize(img, (AXIS_SIZE, AXIS_SIZE, 3))
    return new_img


# Зеркалирование по горизонтали
def flip_img(img):
    new_img = np.fliplr(img)
    return new_img


# Поворот на небольшой угол
def random_rotate_img(img):
    alpha_deg = np.random.randint(-17, 17)
    new_img = rotate(img, alpha_deg, mode='edge')
    return new_img


# Кадрирование
def framing_img(img, j):
    if (j == 0):
        left_edge = int(0.05 * AXIS_SIZE)
        right_edge = int(0.95 * AXIS_SIZE)
    if (j == 1):
        left_edge = int(0.075 * AXIS_SIZE)
        right_edge = int(0.925 * AXIS_SIZE)
    new_img = img[left_edge : right_edge, left_edge : right_edge]
    new_img = resize_img(new_img)
    return new_img


def get_model():
    epochs =  EPOCHS_NUMBER
    batch_sz = BATCHS_NUMBER
    resnet_my = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=(AXIS_SIZE, AXIS_SIZE, 3))
    
    X, y = load_data_to_train(resnet_my)
    
    sz = np.prod(resnet_my.layers[-1].output.shape[1:])
    model_class = Sequential()
    model_class.add(Dense(256, input_dim=int(sz)))
    model_class.add(Dropout(0.4))
    model_class.add(Activation('relu'))
    model_class.add(Dense(50))
    model_class.add(Activation('softmax'))
    
    #callbacks = [ModelCheckpoint('birds_model.hdf5', monitor='val_loss', save_best_only=True)]
    
    model_class.compile(loss='categorical_crossentropy', 
                        optimizer=adam(lr=1e-5), 
                        metrics=['accuracy'])
    '''
    history = model_class.fit(
        X,
        y,
        batch_size=batch_sz,
        epochs=epochs,
        validation_split=0.1,
        verbose=2,
        callbacks=callbacks)
    '''
    
    model_class.fit(X, y, epochs=epochs, batch_size=batch_sz, verbose=1)
    
    model_final = Sequential()
    model_final.add(resnet_my)
    model_final.add(Flatten())
    model_final.add(model_class)

    #model_final.save('med_model.hdf5')
    
    return 


def load_data_to_train(resnet_my):
    old_total = len(art_df)
    total = old_total * 4 * 4 / 5
    sz = np.prod(resnet_my.layers[-1].output.shape[1:])
    sz = int(sz)
    X = (np.zeros((total * sz))).reshape(total, sz)
    y = (np.zeros((total * NUMBER_CLASSES))).reshape(total, NUMBER_CLASSES)
    for i, (filename, class_label) in enumerate(art_df):
        img = imread(filename)
        img = resize_img(img)
        new_img = image.array_to_img(img)
        X[i] = (resnet_my.predict(np.expand_dims(image.img_to_array(new_img), axis=0))).ravel()
        y[i][int(class_label)] = 1
        iid = old_total * 1 + i
        new_img = flip_img(img)
        new_img = image.img_to_array(image.array_to_img(new_img)) # to keras format
        X[iid] = (resnet_my.predict(np.expand_dims(new_img, axis=0))).ravel()
        y[iid][int(class_label)] = 1
        iid = old_total * 2 + i * 1 + j
        new_img = random_rotate_img(img)
        new_img = image.img_to_array(image.array_to_img(new_img)) # to keras format
        X[iid] = (resnet_my.predict(np.expand_dims(new_img, axis=0))).ravel()
        y[iid][int(class_label)] = 1
        iid = old_total * 3 + i * 1 + j
        new_img = framing_img(img, j)
        new_img = image.img_to_array(image.array_to_img(new_img)) # to keras format
        X[iid] = (resnet_my.predict(np.expand_dims(new_img, axis=0))).ravel()
        y[iid][int(class_label)] = 1
        if i + 1 == total:
            break
    return X, y


def load_data_to_test():
    old_total = len(art_df)
    i = old_total * 4 * 4 / 5
    X = np.zeros((total, AXIS_SIZE, AXIS_SIZE, 3))
    while (i < old_total):
        filename = art_df[image_id][i]
        img = imread(filename)
        img = resize_img(img)
        X[i] = image.img_to_array(image.array_to_img(img)) # to keras format
    return X, all_files

In [ ]:
get_model()

TypeError: ignored